# Thể loại phim và phân loại độ tuổi (MPA rating) ảnh hưởng ra sao đến đánh giá của giới phê bình và khán giả?

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

1. Đọc dữ liệu

In [ ]:
try:
    df = pd.read_csv('IMDB_movies.csv')
    
    # Chuẩn hóa tên cột (đề phòng lỗi khoảng trắng/viết hoa)
    df.columns = df.columns.str.strip().str.lower()
    
    required_cols = ['genres', 'mpa', 'metascore', 'rating'] 
    
    # Lọc bỏ các dòng thiếu dữ liệu để vẽ cho đẹp
    df_clean = df[required_cols].dropna().copy()
    
    # Đưa Metascore (0-100) về thang 10 để so sánh với User Rating
    df_clean['metascore_norm'] = df_clean['metascore'] / 10
    
except FileNotFoundError:
    print("Không tìm thấy file csv")
    

2. Xử lí cột genres

In [ ]:
df_exploded = df_clean.assign(genre=df_clean['genre'].str.split(', ')).explode('genre')

3. vẽ biểu đồ

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(25, 12))

BIỂU ĐỒ 1: So sánh khẩu vị Chuyên gia vs Khán giả theo Thể loại

In [ ]:
genre_group = df_exploded.groupby('genre')[['metascore_norm', 'user_rating']].mean().reset_index()
genre_melted = genre_group.melt(id_vars='genre', var_name='Score Type', value_name='Average Score')

sns.barplot(data=genre_melted, x='genre', y='Average Score', hue='Score Type', 
            palette={'metascore_norm': '#E63946', 'user_rating': '#457B9D'}, ax=axes[0])

axes[0].set_title('Sự chênh lệch đánh giá giữa Chuyên gia và Khán giả theo Thể loại', fontsize=14)
axes[0].set_ylabel('Điểm trung bình (Thang 10)')
axes[0].set_xlabel('')
axes[0].tick_params(axis='x', rotation=45)
axes[0].legend(title='Loại điểm', labels=['Chuyên gia (Metascore/10)', 'Khán giả (User Rating)'])
axes[0].grid(axis='y', linestyle='--', alpha=0.5)

BIỂU ĐỒ 2: Phân phối điểm số theo nhãn MPA
Chỉ lấy các nhãn phổ biến: G, PG, PG-13, R, NC-17

In [ ]:
common_mpa = ['G', 'PG', 'PG-13', 'R', 'NC-17']
df_mpa = df_clean[df_clean['mpa'].isin(common_mpa)]

sns.boxplot(data=df_mpa, x='mpa', y='user_rating', order=common_mpa, palette="Set2", ax=axes[1])

axes[1].set_title('Phân phối điểm đánh giá của Khán giả theo phân loại độ tuổi (MPA)', fontsize=14)
axes[1].set_ylabel('Điểm khán giả (User Rating)')
axes[1].set_xlabel('Nhãn MPA')

plt.tight_layout()
plt.savefig('genre_mpa_analysis.png')
plt.show()